# **Code hoàn chỉnh**
```python
from sage.all import *
from pwn import *
from tqdm import tqdm
from collections import Counter

io = process(['python3', 'chall.py'])

F = GF(2**8, 'a')
a = F.gen()

perm1 = []
for i in range(256):
    x = F.from_integer(i)
    y = a * x
    perm1.append(y.to_integer())
perm1[0], perm1[1], perm1[255] = perm1[1], perm1[255], perm1[0]

def fail():
    io.sendline(b'00')
    io.recvuntil(b'Bad luck, try again.')

def try_solve():
    io.sendlineafter(b'Plainperm: ', bytes(perm1).hex().encode())
    io.recvuntil(b'Cipherperm: ')
    cipherperm = bytes.fromhex(io.recvline().strip().decode())
    cipherperm_F = [F.from_integer(x) for x in cipherperm]

    cipherperm_F = [x / (a**63) for x in cipherperm_F]
    for b in range(257):
        assert b < 256, "This should not happen"
        b0 = F.from_integer(b)
        cipherperm_F2 = [(x+b0).to_integer()+1 for x in cipherperm_F]
        if len(Permutation(cipherperm_F2).fixed_points()) > 20:
            break

    perm2 = Permutation(cipherperm_F2)
    cycles = perm2.cycle_tuples()
    cycles = [cycle for cycle in cycles if len(cycle) > 1]
    u = sum([(len(cycle)-1)//2 for cycle in cycles])

    if u != 63 or max([len(cycle) for cycle in cycles]) > 51:
        fail()
        return

    cycles = [tuple(F.from_integer(x-1) for x in cycle) for cycle in cycles]
    print(([len(cycle) for cycle in cycles]), u)

    all_numbers = [F.from_integer(i) for i in range(256)]
    possible_k0 = [all_numbers[:] for _ in range(63)]

    def check(us):
        for cycle in cycles:
            if all(u in cycle for u in us):
                inds = [cycle.index(u) for u in us]
                inversions = 0
                if inds[0] > inds[1]: inversions += 1
                if inds[0] > inds[2]: inversions += 1
                if inds[1] > inds[2]: inversions += 1
                if inversions % 2 == 1:
                    return False
                return True
        return False

    for i in range(63):
        filtered = []
        a0 = a ** i
        ts = [F.from_integer(t)/a0 for t in [0, 1, 255]]

        for k0 in possible_k0[i]:
            us = [u + k0 for u in ts]
            if check(us):
                filtered.append(k0)
        possible_k0[i] = filtered

    def update():
        all_pos = sum([list(cycle) for cycle in cycles], [])
        c = {pos: [] for pos in all_pos}

        for i in range(63):
            a0 = a ** i
            ts = [F.from_integer(t)/a0 for t in [0, 1, 255]]
            for k0 in possible_k0[i]:
                us = [u + k0 for u in ts]
                for u in us:
                    c[u].append((i, k0))

        for pos in c:
            if len(c[pos]) == 1:
                i, k0 = c[pos][0]
                possible_k0[i] = [k0]

    for _ in range(16):
        update()

    if prod([len(x) for x in possible_k0]) > 2**14:
        fail()
        return

    print([len(x) for x in possible_k0], prod([len(x) for x in possible_k0]).bit_length())

    possible_maps = [[] for _ in range(63)]
    for i in range(63):
        a0 = a ** i
        ts = [F.from_integer(t)/a0 for t in [0, 1, 255]]
        for k0 in possible_k0[i]:
            us = [u + k0 for u in ts]
            us = [x.to_integer() + 1 for x in us]
            perm3 = Permutation([tuple(us)])
            possible_maps[i].append((k0, perm3))

    def search(cur_map, i):
        if i == 63:
            if cur_map == perm2:
                return [[]]
            else:
                return None

        all_results = []
        for k0, perm3 in possible_maps[i]:
            new_map = cur_map * perm3
            result = search(new_map, i + 1)
            if result is not None:
                for r in result:
                    all_results.append([k0] + r)

        if all_results:
            return all_results

    pathes = search(cur_map=Permutation(list(range(1, 257))), i=0)
    if pathes is None or len(pathes) == 0:
        fail()
        return

    print(len(pathes), pathes)
    path = pathes[0]

    real_ks = []
    a0 = 1
    c0 = 0
    for i in range(63):
        k0 = path[i]
        ts = [F.from_integer(t)/a0 for t in [0, 1, 255]]
        us = [u + k0 for u in ts]
        c1 = a0 * us[0]
        real_k = (c0 - c1)
        real_ks.append(real_k.to_integer())
        a0 *= a
        c0 += real_k
        c0 *= a
    real_ks.append((b0 * (a ** 63) + c0).to_integer())
    print(bytes(real_ks).hex())
    io.sendlineafter(b'Do you know my key?', bytes(real_ks).hex().encode())
    io.interactive()

for _ in tqdm(range(1000)):
    ret = try_solve()
```

In [74]:
from sage.all import *
from pwn import *
from tqdm import tqdm
from collections import Counter

# # Khởi chạy tiến trình challenge
# io = process(['python3', 'chall.py'])  

-   `F` là trường hữu hạn $ \mathbb{F}_{2^8}$ (sage) với `x` là đại diện biến trong đa thức, làm việc trong trường mở rộng, ta sẽ làm việc với đa thức!
-   `a` là phần tử sinh (hay phần tử nguyên thủy (primitive element)) của trường, dùng làm hệ số nhân để xây ánh xạ tuyến tính.

In [75]:
F = GF(2**8, 'x')
a = F.gen()

print(a.polynomial())       # x

x


-   Ban đầu `perm1` = `a * x` trên mọi giá trị 0..255 (chính là ánh xạ tuyến tính $f(x)=a x$).
-   Sau đó thay đổi để thêm 3-cycle `(0,1,255)`: swap các ảnh tương ứng. Vậy `perm1` = $f \circ g$ với $g=(0,1,255)$. Đây là chính chiến lược trong solution: chỉ thay đổi 3 giá trị để vẫn giữ phần lớn tuyến tính.

In [76]:
perm1 = []
for i in range(256):
    x = F.fetch_int(i) 
    y = a * x
    perm1.append(y.integer_representation())
perm1[0], perm1[1], perm1[255] = perm1[1], perm1[255], perm1[0]

print(perm1[0], perm1[1], perm1[255])
print(perm1)

2 227 0
[2, 227, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100, 102, 104, 106, 108, 110, 112, 114, 116, 118, 120, 122, 124, 126, 128, 130, 132, 134, 136, 138, 140, 142, 144, 146, 148, 150, 152, 154, 156, 158, 160, 162, 164, 166, 168, 170, 172, 174, 176, 178, 180, 182, 184, 186, 188, 190, 192, 194, 196, 198, 200, 202, 204, 206, 208, 210, 212, 214, 216, 218, 220, 222, 224, 226, 228, 230, 232, 234, 236, 238, 240, 242, 244, 246, 248, 250, 252, 254, 29, 31, 25, 27, 21, 23, 17, 19, 13, 15, 9, 11, 5, 7, 1, 3, 61, 63, 57, 59, 53, 55, 49, 51, 45, 47, 41, 43, 37, 39, 33, 35, 93, 95, 89, 91, 85, 87, 81, 83, 77, 79, 73, 75, 69, 71, 65, 67, 125, 127, 121, 123, 117, 119, 113, 115, 109, 111, 105, 107, 101, 103, 97, 99, 157, 159, 153, 155, 149, 151, 145, 147, 141, 143, 137, 139, 133, 135, 129, 131, 189, 191, 185, 187, 181, 183, 177, 179, 173, 175, 169, 171,

-   Gọi khi kiểm tra không thỏa điều kiện (để thoát và thử lại), script gửi `00` làm input (không hợp lệ) để server in `Bad luck, try again.` và script có thể thử lần khác.

In [77]:
# def fail():
#     io.sendline(b'00')
#     io.recvuntil(b'Bad luck, try again.')

### **1.2. Luồng main `try_slove()`:**
**1.** Nhận hoán vị `cipherperm` và chuyển về trường mở rộng $\mathbb{F}_{2^8}$

In [78]:
from numpy import bitwise_xor

plainperm = perm1
key = b'\xb1\x0b,00\x97l\xf2\x82\xa5\xda\xa18\n\xe3_\xe8\x1a\x15\xffWY\xc7\xcc\x8d\xa0Z\xe8\x9cD#\xe20\\m\xb8\xa8\x99g\xedc\x98>"\x96R\xfc\r"\xf9\xed\xeeT\xfe\xa2F\x04\xaem\x9f\xb8\x00t\x92'
print(list(key))

def f(i):
    for k in key[:-1]:
        i = plainperm[bitwise_xor(i, k)]
    return bitwise_xor(i, key[-1])
ciperm = bytes(map(f, range(256)))



# io.sendlineafter(b'Plainperm: ', bytes(perm1).hex().encode())
# io.recvuntil(b'Cipherperm: ')
# cipherperm = bytes.fromhex(io.recvline().strip().decode())
cipherperm = ciperm
cipherperm_F = [F.fetch_int(x) for x in cipherperm]


# print(cipherperm)
# print(cipherperm_F)

[177, 11, 44, 48, 48, 151, 108, 242, 130, 165, 218, 161, 56, 10, 227, 95, 232, 26, 21, 255, 87, 89, 199, 204, 141, 160, 90, 232, 156, 68, 35, 226, 48, 92, 109, 184, 168, 153, 103, 237, 99, 152, 62, 34, 150, 82, 252, 13, 34, 249, 237, 238, 84, 254, 162, 70, 4, 174, 109, 159, 184, 0, 116, 146]


**2.** Chia cho $a^{63}$ để bóc hệ số nhân cuối cùng.\
Trước đó ta biết tổng thể mã hóa có dạng $enc(x) = a^{63} x + S$. Nếu những điểm không bị ảnh hưởng bởi cycle $g(x)$, thì sau khi kết thúc, giá trị sẽ tương đương $enc(x) = a^{63} x + S$ với $S = \sum_{i=1}^{63}{a^{63-i}{k_i}}$.\
Chia từng giá trị cho $a^{63}$ để loại bỏ hệ số nhân, lúc này còn dạng $x + b$ (với một hằng $b$ tùy thuộc vào khóa).\
Mục tiêu là tìm hằng $b'$ sao cho biểu diễn trở nên gần dạng hoán vị có nhiều điểm cố định (vì điểm không bị 3-cycle sẽ thỏa `x -> x` sau điều chỉnh). Tức là, $b + b' \equiv 0$

In [79]:
cipherperm_F = [x / (a**63) for x in cipherperm_F]

# print(cipherperm_F)
# print(cipherperm_F[0].integer_representation())

**3.** Tìm hằng số $b'$

-   Vòng thử `b` từ 0..256 (lệnh `assert` bảo đảm không vì 257 là dư — thực tế loop dừng trước).
-   Tạo `cipherperm_F2` = `(cipherperm_F + b0)` và cộng `+1` vì `Permutation` của sage dùng indexing $1...n$ (không phải $0...n-1$).
-   Lý do kiếm fixed points: nếu ta chọn đúng `b0` tương ứng với hằng $b$ trong $F$, thì mọi điểm *không bị 3-cycle* sẽ trở thành điểm cố định trong `Permutation(cipherperm_F2)`. Vì chỉ ~192 số bị ảnh hưởng, số fixed points lớn (ví dụ >20) báo hiệu tìm được `b0` hợp lý.
-   Khi tìm được `b0` script dừng.

In [80]:
for b in range(257):
    assert b < 256, "This should not happen" # Không có các fixpoint, làm một vòng lặp khác
    b0 = F.fetch_int(b)
    cipherperm_F2 = [(x+b0).integer_representation() + 1 for x in cipherperm_F]
    # print(b, Permutation(cipherperm_F2).fixed_points())
    if len(Permutation(cipherperm_F2).fixed_points()) > 20:
        break
# print(b)

**4.** Lấy chu kỳ của `perm2`
-   `perm2` là hoán vị 1...256 sau bước chuẩn hóa.
-   `cycles` bỏ qua các cycle dài 1 (fixed points).
-   `u = sum((len(cycle)-1)//2)`: Đây là một đại lượng dùng để kiểm tra xem tổng số “3-cycle nguyên tố” có đúng bằng 63 hay không theo cách phân tích (một cycle của độ dài $L$ có thể biểu diễn như một số $(L-1)/2$ các 3-cycle dạng “mở rộng” mà solution đề cập). Script yêu cầu `u == 63` (kì vọng $63$ ba-cycle đã được ghép lại) và chu kỳ không quá dài ($<=51$), nếu không thỏa thì `fail()`.

Một **cycle độ dài $L$** là 
$$(c_1, c_2, \dots, c_L)$$
Nghĩa là hoán vị này gửi: $c_1 \mapsto c_2, c_2 \mapsto c_3, \dots, c_L \mapsto c_1$.


**3-cycle** là cycle chỉ gồm 3 phần tử: $(a,b,c) \implies a\mapsto b, b\mapsto c, c\mapsto a$

-   Mỗi cycle dài $L \ge 3$ có thể **biểu diễn thành chuỗi các 3-cycle** sao cho khi ghép lại, kết quả tương đương với cycle gốc. Ví dụ, cycle 6 có thể được xây thành $(a_1, a_2, a_3), (a_1, a_3, a_4), (a_1, a_4, a_5), (a_1, a_5, a_6)$. 

Theo lý thuyết, với $L \ge 3$, ta có số 3-cycle nguyên tố có thể có là:
$$
\text{round}[(L - 1)/2]
$$

Do đó, ta cần kiểm tra số lượng 3-cycle nguyên tố có thể có phải $ = 63$ (3-cycle)

In [81]:
perm_test = Permutation([2, 3, 1, 5, 4, 6, 9, 8, 10, 11, 7, 12, 13, 14, 15])
print(perm_test)
print(perm_test.cycle_tuples())
# index: 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15
# value: 2  3  1  5  4  6  9  8 10 11  7 12 13 14 15


[2, 3, 1, 5, 4, 6, 9, 8, 10, 11, 7, 12, 13, 14, 15]
[(1, 2, 3), (4, 5), (6,), (7, 9, 10, 11), (8,), (12,), (13,), (14,), (15,)]


In [82]:
perm2 = Permutation(cipherperm_F2)
cycles = perm2.cycle_tuples()
cycles = [cycle for cycle in cycles if len(cycle) > 1] # Loại bỏ các fixed points
u = sum([(len(cycle)-1)//2 for cycle in cycles])
# print(u, cycles)

if u != 63 or max([len(cycle) for cycle in cycles]) > 51: # Phần lớn còn lại: 67 → các cycle phải ngắn ≤ 51
    # fail()
    # return
    pass

**5.** Chuyển các phần tử cycle về $\mathbb{F}_{2^8}$ (vì Permutation là $1...256$) 

In [83]:
cycles = [tuple(F.fetch_int(x-1) for x in cycle) for cycle in cycles] # Do Permutation là 1...256
print(([len(cycle) for cycle in cycles]), u)
# print(cycles)


[5, 7, 15, 81, 3, 3, 3, 5, 5, 3, 3, 3, 3] 63


**5.** Lập danh sách `all_numbers` và danh sách tất cả các giá trị có thể của từng vị trí `k` là `possible_k0`


In [84]:
all_numbers = [F.fetch_int(i) for i in range(256)]
possible_k0 = [all_numbers[:] for _ in range(63)]

print(all_numbers)
print(possible_k0)

[0, 1, x, x + 1, x^2, x^2 + 1, x^2 + x, x^2 + x + 1, x^3, x^3 + 1, x^3 + x, x^3 + x + 1, x^3 + x^2, x^3 + x^2 + 1, x^3 + x^2 + x, x^3 + x^2 + x + 1, x^4, x^4 + 1, x^4 + x, x^4 + x + 1, x^4 + x^2, x^4 + x^2 + 1, x^4 + x^2 + x, x^4 + x^2 + x + 1, x^4 + x^3, x^4 + x^3 + 1, x^4 + x^3 + x, x^4 + x^3 + x + 1, x^4 + x^3 + x^2, x^4 + x^3 + x^2 + 1, x^4 + x^3 + x^2 + x, x^4 + x^3 + x^2 + x + 1, x^5, x^5 + 1, x^5 + x, x^5 + x + 1, x^5 + x^2, x^5 + x^2 + 1, x^5 + x^2 + x, x^5 + x^2 + x + 1, x^5 + x^3, x^5 + x^3 + 1, x^5 + x^3 + x, x^5 + x^3 + x + 1, x^5 + x^3 + x^2, x^5 + x^3 + x^2 + 1, x^5 + x^3 + x^2 + x, x^5 + x^3 + x^2 + x + 1, x^5 + x^4, x^5 + x^4 + 1, x^5 + x^4 + x, x^5 + x^4 + x + 1, x^5 + x^4 + x^2, x^5 + x^4 + x^2 + 1, x^5 + x^4 + x^2 + x, x^5 + x^4 + x^2 + x + 1, x^5 + x^4 + x^3, x^5 + x^4 + x^3 + 1, x^5 + x^4 + x^3 + x, x^5 + x^4 + x^3 + x + 1, x^5 + x^4 + x^3 + x^2, x^5 + x^4 + x^3 + x^2 + 1, x^5 + x^4 + x^3 + x^2 + x, x^5 + x^4 + x^3 + x^2 + x + 1, x^6, x^6 + 1, x^6 + x, x^6 + x + 1,

#### ***1.2.1. Phương thức `checkus()` kiểm tra điều kiện thứ tự trong một cycle***
-   `us` là 3 phần tử (ứng với `u + k0` cho ba giá trị $0,1,255$ sau tác động của vòng $i$).
    
-   `check` tìm `cycle` chứa cả ba phần tử; nếu có thì kiểm tra *parity of inversions* giữa chỉ số của ba phần tử trong cycle. Tại sao? Vì một 3-cycle có cấu trúc circular ordering, chỉ một trong hai hoán vị thứ tự chấp nhận — condition parity đảm bảo thứ tự là hợp lệ cho dạng 3-cycle được giả sử. (Nói ngắn: đảm bảo ba phần tử có thứ tự tương thích với dạng 3-cycle xây dựng.)
-   **Tóm lại:** Mục đích là kiểm tra `us` có phải là một cycle hợp lệ hay không, một cycle hợp lệ khi thứ tự của chúng tăng dần hoặc các cặp thứ tự đảo ngược của chúng luôn là một số chẵn

Ví dụ, `cycle = (1,3,5,7)` và `us = (3,1,7)` → `inds = [1,0,3]` -> `False`, `3 -> 7` mới đúng. Ngược lại, ví dụ `us = (1, 5, 3)` → `inds = [0, 2, 1]` -> `inversions = 2` -> `True`.

In [85]:
def check(us):
    for cycle in cycles:
        if all(u in cycle for u in us):
            inds = [cycle.index(u) for u in us]
            inversions = 0
            if inds[0] > inds[1]: inversions += 1
            if inds[0] > inds[2]: inversions += 1
            if inds[1] > inds[2]: inversions += 1
            if inversions % 2 == 1:
                return False
            return True
    return False

**6.** Lọc `possible_k0` dựa trên `check`
- ***Mục đích:*** Xác định các giá trị `k0` hợp lệ dựa trên cấu trúc 3-cycle, từ đó dần dần suy ra key thực của hệ thống.

- ***Ý tưởng:*** 
    - Trong mô hình, ta luôn có 3-cycle tại 3 vị trí của `plainperm` gồm $(0, 1, 255)$. Do đó, các giá trị $x$ của $g_i(x)$ thuộc ba giá trị trên. Tương đương $g_{i-1}(x) \in {0, 1, 255}$. 
    - Với mỗi vòng, trước khi đến $g_i(x)$, ta phải đưa giá trị qua hàm $f_i$. Do đó, ta phải có tuyến tính, tức là $f_{i}(x+k_i) = \alpha$ và $\alpha = {0, 1, 255}$ trước khi đưa vào $g_i$.
    - Vì mỗi cycle sẽ có 3 giá trị riêng biệt, do đó, mỗi giá trị đầu ra chỉ có 1 cycle hoạt động. Do đó, nó chỉ đi qua 1 hàm $g_i$ nào đó, trước khi đi vào hàm $g_i$, nó sẽ đi qua tất cả hàm, vì vậy sẽ có dạng $a^ix + \beta_i$. Mục đích hiện tại là tìm $\beta_i$ cho từng trường hợp, sau đó, sử dụng đệ quy + backtracking để tìm ra $k_i$. 

- ***Quy trình:*** 
    -   Với mỗi vòng $i$, ta tính `ts = [0,1,255] / a^i` vì khi đẩy ngược nhân theo vòng ta cần chia cho $a^i$.
    
    -   Với mỗi ứng viên `k0`, ta xem ba vị trí sau `+ k0` có rơi vào cùng một cycle với thứ tự hợp lệ không. Nếu có, giữ `k0`.
    
    -   Đây là bước loại bỏ đại số dựa trên chu kỳ thực tế quan sát được.

In [86]:
for i in range(63):
    filtered = []
    a0 = a ** i
    ts = [F.fetch_int(t)/a0 for t in [0, 1, 255]]
    print(ts)
    for k0 in possible_k0[i]:
        us = [u + k0 for u in ts]
        if check(us):
            filtered.append(k0)
    possible_k0[i] = filtered               # Các giá trị beta(b0) khả dĩ

[0, 1, x^7 + x^6 + x^5 + x^4 + x^3 + x^2 + x + 1]
[0, x^7 + x^3 + x^2 + x, x^7 + x^6 + x^5 + x^4 + 1]
[0, x^6 + x^2 + x + 1, x^7 + x^6 + x^5 + x^4 + x^2 + x]
[0, x^7 + x^5 + x^3 + x^2 + 1, x^6 + x^5 + x^4 + x^3 + x + 1]
[0, x^7 + x^6 + x^4 + x^3, x^7 + x^5 + x^4 + x + 1]
[0, x^6 + x^5 + x^3 + x^2, x^7 + x^6 + x^4 + x^2 + x + 1]
[0, x^5 + x^4 + x^2 + x, x^7 + x^6 + x^5 + x^2 + 1]
[0, x^4 + x^3 + x + 1, x^7 + x^6 + x^5 + x^4 + x^3 + x^2]
[0, x^7 + x + 1, x^6 + x^5 + x^4 + x^3 + x^2 + x]
[0, x^7 + x^6 + x^3 + x^2 + x + 1, x^5 + x^4 + x^3 + x^2 + x + 1]
[0, x^7 + x^6 + x^5 + x^3 + 1, x^7 + x^4 + 1]
[0, x^7 + x^6 + x^5 + x^4 + x^3 + x, x^7 + x^6 + x^2 + x]
[0, x^6 + x^5 + x^4 + x^3 + x^2 + 1, x^6 + x^5 + x + 1]
[0, x^7 + x^5 + x^4, x^7 + x^5 + x^4 + x^3 + x^2 + x + 1]
[0, x^6 + x^4 + x^3, x^7 + x^6 + x^4 + 1]
[0, x^5 + x^3 + x^2, x^7 + x^6 + x^5 + x^2 + x]
[0, x^4 + x^2 + x, x^6 + x^5 + x^4 + x + 1]
[0, x^3 + x + 1, x^7 + x^5 + x^4 + x^2 + x + 1]
[0, x^7 + x^3 + x + 1, x^7 + x^6 + x^4 + x^2

#### ***1.2.2. Phương thức `update()` để cô lập các lựa chọn đơn nghĩa***
- **Mục đích:** Xác định những điểm cycle có thể được tạo từ vòng lặp $i$ và tham số $\beta_i$. Trên cơ sở đó, những điểm được tạo bởi duy nhất một cặp $(\beta_i, i)$ thì điểm đó chính xác được tạo từ cặp đó.

- **Quy trình:** 
    - Đầu tiên, nối tất cả các phần tử trong mọi cycle thành một danh sách chung `all_pos`, dạng field element.
    - Tiếp theo, tạo từ điển `c`, ánh xạ mỗi vị trí `pos` → danh sách rỗng `[]`, sẽ được dùng để ghi lại những vòng $i$ và khóa $k0$ nào có thể tạo ra `pos` này.
    -   Với mỗi vòng $i$, tính `a0 = a^i`, rồi “chuẩn hóa” 3 điểm tham chiếu `[0,1,255]` bằng phép chia `a0`. Với mỗi khóa khả thi `k0` của vòng đó, ta **tính ảnh thực tế của các điểm** sau khi cộng thêm `k0`: $u = t/a^i + k_0$. Mỗi `u` này đại diện cho một **vị trí trong hoán vị có thể do (i, k0) sinh ra**.
    -   Ta lưu lại thông tin `(i, k0)` vào danh sách `c[u]` → Nếu nhiều vòng $i$ hoặc nhiều giá trị $k0$ khác nhau cho ra cùng một `u`, thì `c[u]` sẽ có nhiều phần tử.
    -   Nếu tại một vị trí `pos`, chỉ có đúng **một cặp (i, k0)** có thể sinh ra nó,  
    → thì chắc chắn `k0` là **khóa thực tế** của vòng `i`.
    -   Cập nhật: Đặt `possible_k0[i] = [k0]` (chỉ còn một giá trị duy nhất).

In [87]:
def update():
    all_pos = sum([list(cycle) for cycle in cycles], [])        # Concat all cycle element to 1 list
    c = {pos: [] for pos in all_pos}

    for i in range(63):
        a0 = a ** i
        ts = [F.fetch_int(t)/a0 for t in [0, 1, 255]]
        for k0 in possible_k0[i]:
            us = [u + k0 for u in ts]
            for u in us:
                c[u].append((i, k0))

    for pos in c:
        if len(c[pos]) == 1:
            i, k0 = c[pos][0]
            possible_k0[i] = [k0]

- Thực hiện 16 lần `update()` để lan toả ràng buộc (propagation): một số biến sẽ được cố định và giúp cố định các biến khác.

In [88]:
# print(possible_k0)
for _ in range(16):
    update()
    # print(possible_k0)

**7.** Kiểm tra kích thước không gian khả dĩ của `k0`
-   Nếu không gian tổ hợp `len(possible_k0[i])` quá lớn (> 2^14) thì bỏ (quá tốn thời gian), script trả `fail()` để thử lần khác. Đây là cắt nhánh thực tế để tránh khám phá quá nhiều.

In [89]:
if prod([len(x) for x in possible_k0]) > 2**14: #prod([a, b, c]) = a*b*c
    # fail()
    # return
    pass

print([len(x) for x in possible_k0], prod([len(x) for x in possible_k0]).bit_length())

[1, 4, 7, 1, 1, 6, 8, 1, 2, 5, 1, 1, 1, 3, 4, 1, 7, 3, 5, 6, 1, 4, 1, 3, 6, 1, 3, 6, 1, 3, 3, 6, 1, 1, 5, 8, 1, 5, 1, 4, 3, 1, 1, 3, 4, 5, 4, 1, 3, 4, 6, 1, 1, 3, 2, 5, 1, 1, 8, 4, 6, 6, 1] 84


**8.** Xây `possible_maps`, ánh xạ 3-cycle tương ứng cho mỗi vòng
-   **Mục đích:** Tại mỗi vòng `i`, ta xây dựng một **tập các ánh xạ (nhỏ) có thể** mà vòng đó gây ra lên ba điểm kiểm tra `[0,1,255]` với mọi `k0` khả dĩ. Mỗi phần tử của `possible_maps[i]` là một cặp `(k0, perm3)`:
    -   `k0`: Một ứng viên byte khóa cho vòng `i`
    -   `perm3`: Một hoán vị nhỏ (được tạo từ ba ảnh của `[0,1,255]` sau khi chia `a^i` và cộng `k0`), biểu diễn “cách vòng $i$ dịch ba điểm kiểm tra”
-   **Ý tưởng:** Sau khi thu được `possible_maps` cho mọi `i`, ta có thể thử kết hợp (compose) các hoán vị nhỏ này theo thứ tự để xem có tái tạo được `cipherperm` hay không, sử dụng không gian các perm khả thi để dựng lại hoán vị tổng thể.

In [90]:
pe_test = Permutation([(10,3,2)])
print(pe_test)

[1, 10, 2, 4, 5, 6, 7, 8, 9, 3]


In [91]:
possible_maps = [[] for _ in range(63)]
for i in range(63):
    a0 = a ** i
    ts = [F.fetch_int(t)/a0 for t in [0, 1, 255]]
    for k0 in possible_k0[i]:
        us = [u + k0 for u in ts]
        us = [x.integer_representation() + 1 for x in us]
        perm3 = Permutation([tuple(us)])
        possible_maps[i].append((k0, perm3))
# for i in possible_maps:
#     print(i)   

#### ***1.2.3. Phương thức `search()` Backtracking tìm chuỗi 63 3-cycle khớp `perm2`***
- **Mục đích:** **Đệ quy thử tất cả các khả năng** cho từng vòng `i = 0..62`, với mục tiêu là tìm một **chuỗi khóa vòng** `k0[0], k0[1], …, k0[62]` sao cho tích các hoán vị tương ứng (`perm3`) **bằng đúng hoán vị đích `perm2`** (tức là hoán vị đã quan sát hoặc được mã hóa)..

In [92]:
def search(cur_map, i):
    if i == 63:
        if cur_map == perm2:
            return [[]]
        else:
            return None

    all_results = []
    for k0, perm3 in possible_maps[i]:
        new_map = cur_map * perm3
        result = search(new_map, i + 1)
        if result is not None:
            for r in result:
                all_results.append([k0] + r)

    if all_results:
        return all_results

In [ ]:
pathes = search(cur_map=Permutation(list(range(1, 257))), i=0)
print(pathes)
if pathes is None or len(pathes) == 0:
    # fail()
    # return
    pass


In [ ]:
print(len(pathes), pathes)
path = pathes[0]

real_ks = []
a0 = 1
c0 = 0
for i in range(63):
    k0 = path[i]
    ts = [F.fetch_int(t)/a0 for t in [0, 1, 255]]
    us = [u + k0 for u in ts]
    c1 = a0 * us[0]
    real_k = (c0 - c1)
    real_ks.append(real_k.integer_representation())
    a0 *= a
    c0 += real_k
    c0 *= a
real_ks.append((b0 * (a ** 63) + c0).integer_representation())
print(bytes(real_ks).hex())
# io.sendlineafter(b'Do you know my key?', bytes(real_ks).hex().encode())
# io.interactive()

# for _ in tqdm(range(1000)):
#     ret = try_solve()